## Understanding an MNIST model in Keras end-to-end

In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
batch_size = 64
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

15220736/15296311 [============================>.] - ETA: 0s 

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [4]:
X_test = np.expand_dims(X_test, 1)
X_train = np.expand_dims(X_train, 1)

X_test.shape

(10000, 1, 28, 28)

In [5]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [7]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [8]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [9]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [10]:
def norm_input(x):
    return (x-mean_px)/std_px

## Linear Model

In [24]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [25]:
lm = get_lin_model()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [26]:
gen = image.ImageDataGenerator()
# Generating augmented images of the dataset
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [27]:
lm.fit_generator(batches, batches.N, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.4273 - acc: 0.8750 - val_loss: 0.3039 - val_acc: 0.9135  192/60000 [..............................] - ETA: 17s - loss: 3.2432 - acc: 0.0573

In [29]:
lm.optimizer.lr = 0.1

In [30]:
lm.fit_generator(batches, batches.N, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.2992 - acc: 0.9148 - val_loss: 0.2830 - val_acc: 0.9191  128/60000 [..............................] - ETA: 13s - loss: 0.2650 - acc: 0.9219

In [31]:
lm.optimizer.lr = 0.01

In [32]:
lm.fit_generator(batches, batches.N, nb_epoch=4, validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 12s - loss: 0.2849 - acc: 0.9195 - val_loss: 0.2902 - val_acc: 0.9152  128/60000 [..............................] - ETA: 13s - loss: 0.2228 - acc: 0.9219

## Above, the code network was a linear model.

## Below, I will add a dense(fully connected) layer

In [ ]:
def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape(1,28,28)),
        Flatten(),
        Dense(512, activation='softmax'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
fc = get_fc_model()